In [1]:
class Treenode():
    def __init__(self,feat,val,right,left):
        featuretospliton=feat
        valueofsplit=val
        righttree=right
        lefttree=left

In [2]:
import numpy as np

#加载数据
def loadDataset(filename):
    dataMat=[]
    fr=open(filename)
    for line in fr.readlines():
        curline=line.strip().split('\t')
        dataMat.append(map(float,curline))
    return dataMat

#根据值分割数据集
def binsplitDataset(dataset,feature,value):
    try:
        mat0=dataset[np.nonzero(dataset[:,feature]>value)[0],:]
        mat1=dataset[np.nonzero(dataset[:,feature]<=value)[0],:]
    except:
        mat0=[]
        mat1=[]
    return mat0, mat1

#创建树
def createTree(dataset,ops=(1,4)):
    feat,val=chooseBestSplit(dataset,ops)
    if feat==None:
        return val
    reTree={}
    reTree['feat']=feat
    reTree['value']=val
    lSet,rSet=binsplitDataset(dataset,feat,val)
    reTree['left']=createTree(lSet,ops)
    reTree['right']=createTree(rSet,ops)
    return reTree

In [3]:
testMat=np.mat(np.eye(4))
mat0,mat1=binsplitDataset(testMat,1,0.5)
print binsplitDataset(testMat,1,0.5)

(matrix([[ 0.,  1.,  0.,  0.]]), matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]))


In [4]:
#计算总平方差
def regErr(dataset):
    return np.var(dataset[:,-1])*dataset.shape[0]
#选择最佳的分割特征和特征值
def chooseBestSplit(dataset,ops=(1,4)):
    tolS=ops[0];tolN=ops[1]
    lowestErr=np.inf
    bestfeat=None;bestVal=None
    #如果所有值都相同停止分割
    if len(set(dataset[:,-1].T.tolist()[0]))==1:
        return None,np.mean(dataset[:,-1])
    for feat in range(dataset.shape[1]-1):
        values=set(dataset[:,feat].T.tolist()[0])
        for value in values:
            mat0,mat1=binsplitDataset(dataset,feat,value)
            #如果分割后数目少于最低要求跳过该循环
            if mat0.shape[0]<tolN or mat1.shape[0]<tolN:
                continue
            Err=regErr(mat0)+regErr(mat1)
            if Err<lowestErr:
                bestfeat=feat
                bestVal=value
                lowestErr=Err
    splitdata0,splitdata1=binsplitDataset(dataset,bestfeat,bestVal)
    #如果与之前Error相比在限差之内则不进行更新
    if regErr(dataset)-lowestErr<tolS:
        return None,np.mean(dataset[:,-1])
    if splitdata0.shape[0]<tolN or splitdata1.shape[0]<tolN:
        return None,np.mean(dataset[:,-1])
    return bestfeat,bestVal

In [5]:
mydata=np.mat(loadDataset('ex00.txt'))
print createTree(mydata,ops=(1,4))

{'right': -0.044650285714285719, 'feat': 0, 'value': 0.48813, 'left': 1.0180967672413792}


In [6]:
mydata2=np.mat(loadDataset('ex0.txt'))
print createTree(mydata2,ops=(1,4))

{'right': {'right': -0.023838155555555553, 'feat': 1, 'value': 0.197834, 'left': 1.0289583666666666}, 'feat': 1, 'value': 0.39435, 'left': {'right': 1.980035071428571, 'feat': 1, 'value': 0.582002, 'left': {'right': 2.9836209534883724, 'feat': 1, 'value': 0.797583, 'left': 3.9871631999999999}}}


In [7]:
#树剪枝

#取左右节点的平均值
def getMean(tree):
    if type(tree['left']).__name__=='dict':
        tree['left']=getMean(tree['left'])
    if type(tree['right']).__name__=='dict':
        tree['right']=getMean(tree['right'])
    return (tree['left']+tree['right'])/2.0

#后剪枝
def prune(tree,testData):
    if testData.shape[1]==0:
        return getMean(tree)
    lset,rset=binsplitDataset(testMat,tree['feat'],tree['value'])
    if type(tree['left']).__name__=='dict':
        tree['left']=prune(tree['left'],lset)
    if type(tree['right']).__name__=='dict':
        tree['right']=prune(tree['right'],rset)
    if type(tree['left']).__name__!='dict' and type(tree['right']).__name__!='dict':
        errnomerge=np.sum(np.power(lset[:,-1]-tree['left'],2))+np.sum(np.power(rset[:,-1]-tree['right'],2))
        errmerge=np.sum(np.power(testData[:,-1]-getMean(tree),2))
        if errmerge<errnomerge:
            print 'merging'
            return getMean(tree)
        else:
            return tree
    else:
        return tree

In [8]:
myMat2=np.mat(loadDataset('ex2.txt'))
mytree=createTree(myMat2,(0,1))
print prune(mytree,np.mat(loadDataset('ex2test.txt')))

merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging
merging


In [9]:
def linearSolve(dataset):
    m,n=np.shape(dataset)
    X=np.mat(np.ones((m,n)))
    Y=np.mat(np.ones((m,1)))
    X[:,1:n]=dataset[:,0:n-1]
    Y=dataset[:,-1]
    if np.linalg.det(X.T*X)==0:
        print 'matrix cannot be inverse'
        return
    para=(X.T*X).I*X.T*Y
    return para,X,Y

def reglinearLeaf(dataset):
    para,X,Y=linearSolve(dataset)
    return para

def reglinearErr(dataset):
    para,X,Y=linearSolve(dataset)
    return np.sum(np.power(X*para-Y,2))

#选择最佳的分割特征和特征值
def chooseLinearBestSplit(dataset,ops=(1,4)):
    tolS=ops[0];tolN=ops[1]
    lowestErr=np.inf
    bestfeat=None;bestVal=None
    #如果所有值都相同停止分割
    if len(set(dataset[:,-1].T.tolist()[0]))==1:
        return None,reglinearLeaf(dataset)
    for feat in range(dataset.shape[1]-1):
        values=set(dataset[:,feat].T.tolist()[0])
        for value in values:
            mat0,mat1=binsplitDataset(dataset,feat,value)
            #如果分割后数目少于最低要求跳过该循环
            if mat0.shape[0]<tolN or mat1.shape[0]<tolN:
                continue
            Err=reglinearErr(mat0)+reglinearErr(mat1)
            if Err<lowestErr:
                bestfeat=feat
                bestVal=value
                lowestErr=Err
    splitdata0,splitdata1=binsplitDataset(dataset,bestfeat,bestVal)
    #如果与之前Error相比在限差之内则不进行更新
    if reglinearErr(dataset)-lowestErr<tolS:
        return None,reglinearLeaf(dataset)
    if splitdata0.shape[0]<tolN or splitdata1.shape[0]<tolN:
        return None,reglinearLeaf(dataset)
    return bestfeat,bestVal

def createLinearTree(dataset,ops=(1,4)):
    feat,val=chooseLinearBestSplit(dataset,ops)
    if feat==None:
        return val
    reTree={}
    reTree['feat']=feat
    reTree['value']=val
    lSet,rSet=binsplitDataset(dataset,feat,val)
    reTree['left']=createLinearTree(lSet,ops)
    reTree['right']=createLinearTree(rSet,ops)
    return reTree

In [10]:
myMat2=np.mat(loadDataset('exp2.txt'))
createLinearTree(myMat2,(1,10))

{'feat': 0, 'left': matrix([[  1.69855694e-03],
         [  1.19647739e+01]]), 'right': matrix([[ 3.46877936],
         [ 1.18521743]]), 'value': 0.285477}

In [11]:
testMat=np.mat(np.eye(4))
mat0,mat1=binsplitDataset(testMat,1,0.5)
print binsplitDataset(testMat,1,0.5)

(matrix([[ 0.,  1.,  0.,  0.]]), matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]))


In [13]:
import Tkinter as tk
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure


def indexTree(tree,inputx,treetype):
    if type(tree).__name__=='dict':
        if inputx[0,tree['feat']]>tree['value']:
            return indexTree(tree['left'],inputx,treetype)
        if inputx[0,tree['feat']]<=tree['value']:
            return indexTree(tree['right'],inputx,treetype)
    else:
        if treetype=='model':
            n=np.shape(inputx)[1]
            X=np.mat(np.ones((1,n+1)))
            X[:,1:n+1]=inputx
            return float(X*tree)
        if treetype=='regtree':
            return float(tree)
    
def createForeTree(dataset,tree,treetype):
    yhat=np.mat(np.zeros((dataset.shape[0],1)))
    for i in range(dataset.shape[0]):
        yhat[i,0]=indexTree(tree,np.mat(dataset[i]),treetype)
    return yhat
        
def reDraw(tolS,tolN):
    reDraw.f.clf()
    reDraw.a=reDraw.f.add_subplot(111)
    if chkBtnVar.get():
        if tolN<2:tolN=2
        mytree=createLinearTree(reDraw.rawData,ops=(tolS,tolN))
        yhat=createForeTree(reDraw.testData,mytree,'model')
    else:
        mytree=createTree(reDraw.rawData,ops=(tolS,tolN))
        yhat=createForeTree(reDraw.testData,mytree,'regtree')
    reDraw.a.scatter(reDraw.rawData[:,0].tolist(),reDraw.rawData[:,1].tolist(),s=5)
    reDraw.a.plot(reDraw.testData,yhat,linewidth=2.0)
    reDraw.canvas.show()

def getInput():
    try:
        tolN=int(tolNentry.get())
    except:
        tolN=10
        print 'Please input int'
        tolNentry.delete(0,END)
        tolNentry.insert(0,'10')
    try:
        tolS=float(tolSentry.get())
    except:
        tolS=1.0
        print 'Please input float'
        tolNentry.delete(0,END)
        tolNentry.insert(0,'1.0')
    return tolN,tolS

def drawNewTree():
    tolN,tolS=getInput()
    reDraw(tolS,tolN)
    
root=tk.Tk()
reDraw.f=Figure(figsize=(5,4),dpi=100)
reDraw.canvas=FigureCanvasTkAgg(reDraw.f,master=root)
reDraw.canvas.show()
reDraw.canvas.get_tk_widget().grid(row=0,columnspan=3)

tk.Label(root,text='tolN').grid(row=1,column=0)
tk.Label(root,text='tols').grid(row=2,column=0)
tolNentry=tk.Entry(root)
tolNentry.grid(row=1,column=1)
tolNentry.insert(0,'10')
tolSentry=tk.Entry(root)
tolSentry.grid(row=2,column=1)
tolSentry.insert(0,'1.0')
tk.Button(root,text='Redraw',command=drawNewTree).grid(row=1,column=2,rowspan=2)
chkBtnVar=tk.IntVar()
chkBtn=tk.Checkbutton(root,text='Model Tree',variable=chkBtnVar)
chkBtn.grid(row=3,column=1)
reDraw.rawData=np.mat(loadDataset('sine.txt'))
reDraw.testData=np.arange(np.min(reDraw.rawData[:,0]),np.max(reDraw.rawData[:,0]),0.01)
reDraw(1.0,10)
root.mainloop()
